# Load modules and model

In [1]:
import cobra.test
from cobra.medium import minimal_medium
from cobra import Model, Reaction, Metabolite

#load the model
model = cobra.io.read_sbml_model('Paracoccus_MinimalMedia_exchanges.sbml')

# Add sMMO reaction

In [2]:
# Add sMMO from Methylococcus Capsulatus & methane uptake in Paracoccus
# NADH + O2 + H+ + Methane => H2O + NAD + Methanol
# cpd00004 + cpd00007 + cpd00067 + cpd01024 => cpd00001 + cpd00003 + cpd00116
rxn00843_c0 = Reaction('rxn00843_c0')
rxn00843_c0.name = 'Methane,NADH:oxygen oxidoreductase (hydroxylating) (sMMO)'
rxn00843_c0.lower_bound = 0.0
rxn00843_c0.upper_bound = 1000.0

cpd00004_c0 = model.metabolites.get_by_id('cpd00004_c0')
cpd00007_c0 = model.metabolites.get_by_id('cpd00007_c0')
cpd00067_c0 = model.metabolites.get_by_id('cpd00067_c0')
cpd01024_c0 = Metabolite(
    'cpd01024_c0', 
    formula = 'CH4', 
    name = 'Methane_c0',
    compartment = 'c0')
cpd00001_c0 = model.metabolites.get_by_id('cpd00001_c0')
cpd00003_c0 = model.metabolites.get_by_id('cpd00003_c0')
cpd00116_c0 = model.metabolites.get_by_id('cpd00116_c0')

rxn00843_c0.add_metabolites({
    cpd00004_c0: -1.0,
    cpd00007_c0: -1.0,
    cpd00067_c0: -1.0,
    cpd01024_c0: -1.0,
    cpd00001_c0: 1.0,
    cpd00003_c0: 1.0,
    cpd00116_c0: 1.0
})

# Methane transport via diffusion
# Methane[1] <=> Methane
# cpd01024[1] <=> cpd01024
rxn10471_c0 = Reaction('rxn10471_c0')
rxn10471_c0.name = 'Methane,NADH:oxygen oxidoreductase (hydroxylating) (sMMO)'
rxn10471_c0.lower_bound = -1000.0
rxn10471_c0.upper_bound = 1000.0

cpd01024_e0 = Metabolite(
    'cpd01024_e0', 
    formula = 'CH4', 
    name = 'Methane_e0',
    compartment = 'e0')

rxn10471_c0.add_metabolites({
    cpd01024_e0: -1.0,
    cpd01024_c0: 1.0
})

# Methane exhange
model.add_boundary(cpd01024_e0, type="exchange", reaction_id="EX_cpd01024_e0", ub=1000.)


model.add_reactions([rxn00843_c0, rxn10471_c0])

In [3]:
# Change C-source
medium = model.medium
medium['EX_cpd01024_e0'] = 11.16 # methane > achieve max growth rate of 0.16 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC209051/pdf/jbacter01039-0333.pdf
medium['EX_cpd00116_e0'] = 0.0 # methanol
medium['EX_cpd00011_e0'] = 0.0 # CO2
model.medium = medium

# Check and save the model

In [4]:
# Run to check with FBA if the reactions work
model.optimize()

,fluxes,reduced_costs
rxn02201_c0,0.00,2.220446e-16
rxn00351_c0,0.00,-5.773160e-15
rxn07431_c0,0.00,5.551115e-17
rxn00390_c0,0.00,0.000000e+00
rxn08180_c0,0.00,0.000000e+00
...,...,...
rxn08461_c0,0.00,-0.000000e+00
rxn05703_c0,0.00,0.000000e+00
EX_cpd01024_e0,-11.16,-2.867464e-02
rxn00843_c0,11.16,2.220446e-16


In [5]:
# Save the engineered model >> set save value to True
save = False

if save == True:
    cobra.io.write_sbml_model(model, "Paracoccus_MinimalMedia_sMMO.sbml")